In [1]:
import scipy.io as sio
from scipy.misc import imresize
from PIL import Image
import numpy as np
import glob
import os
import cv2
from skimage import transform as tf

In [2]:
#-----------------------------
#paths for feature maps input
#-----------------------------

#relu5_maps
fmap_path = 'E:/blk_5_maps';

#SSNet Probability Maps
detection_maps_path = 'E:/ssnet_maps';

#-----------------------------
# paths for fused maps output
#-----------------------------
#product maps
output_path_pfmaps = 'E:/pf_maps';

#Global Average Pooled Vectors
output_path_pgap_maps = 'E:/pgap_maps';

#load file names
fmaps_filenames = glob.glob(fmap_path+"/*.mat")
dmaps_filenames = glob.glob(detection_maps_path+"/*.mat")

In [1]:
sz = 42 #resize value

for i in range(len(fmaps_filenames)):
    
    print(i)
    
    #load maps
    fmap = sio.loadmat(fmaps_filenames[i])
    fmap = fmap["fmap"]
    fmap = np.squeeze(fmap, axis=0)
    
    #bilinear interpolation
    fmap_interpolated = np.zeros((sz,sz,fmap.shape[2]))
    for j in range(fmap.shape[2]):
        fmap_interpolated[:,:,j] = cv2.resize(fmap[:,:,j],(sz,sz))
    
    dmap = sio.loadmat(dmaps_filenames[i])
    dmap = dmap["Prob"]

    dmap = cv2.resize(dmap, (sz,sz), interpolation = cv2.INTER_LINEAR)

    pfmap = np.zeros((42,42,1024))
    pgap = np.zeros((1,1024))
    
    for k in range(1024):
        pfmap[:,:,k] = np.dot(fmap_interpolated[:,:,k],dmap)
        pgap[0,k] = np.sum(pfmap[:,:,k])/np.prod(pfmap[:,:,k].shape)

    pgap = pgap.astype('float32')
    pfmap = pfmap.astype('float32')
    
    #saving mat files to output paths
    sio.savemat(output_path_pfmaps + "/" + os.path.basename(fmaps_filenames[i]), {"pfmap": pfmap})
    sio.savemat(output_path_pgap_maps + "/" + os.path.basename(fmaps_filenames[i]), {"Pgap": pgap})
